In [41]:

# Example usage
if __name__ == "__main__":
    system_message = "You are a professional debator named Sam. You are arguing against the motion that 'The internet is a force for good'. Refer to Bob's points when arguing. The messages provided to you will be Bob's arguments."
    debate_agent = DebateAgent(system_message)

    # Add a human message
    debate_agent.add_human_message("Bob: The internet is a force of good because it enables global communication, provides access to information, fosters innovation, and connects people, facilitating collaboration and knowledge sharing across diverse communities.")

    # Get the AI response
    ai_response = debate_agent.get_ai_response()
    print("AI Response:", ai_response)
    
    # Print chat history
    for message in debate_agent.get_chat_history():
        print(message)

AI Response: As a professional debater, I disagree with the notion that the internet is solely a force for good. While Bob makes valid points about the positive aspects of the internet, such as enabling global communication and providing access to information, there are significant drawbacks and negative consequences that cannot be ignored.

1. **Misinformation and Fake News**: The internet is plagued with misinformation and fake news, which can spread rapidly and have harmful consequences. People can easily be misled by false information, leading to confusion, division, and even public health risks.

2. **Privacy Concerns**: The internet poses serious privacy concerns as personal data is often collected, stored, and shared without users' consent. This lack of privacy can lead to identity theft, cyberbullying, and surveillance by governments and corporations.

3. **Digital Divide**: Despite the internet's potential to provide access to information, there is a significant digital divide